In [1]:
from conf import *
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix

## Reading CSV

In [2]:
df = pd.read_csv('dataset/dataset2.csv')
df.head(4)

,ROLLNO,STUDENT_NAME,BE103_First,BE110_First,BE101-05_First,CS110_First,EE100_First,EE110_First,MA101_First,PH100_First,...,CS409_Seccond,ELECTIVE3_Second,SGPA_S1,SGPA_S2,SGPA_S3,SGPA_S4,SGPA_S5,SGPA_S6,Unnamed: 107,Unnamed: 108
0,1.0,ABHIN JOY J A,55.0,45.0,80.0,47.0,70.0,57.0,65.0,45.0,...,25.0,45.0,7.00,0.00,0.00,0.00,0.00,6.22,NaN,NaN
1,2.0,AHKIN JACOB,60.0,93.0,60.0,57.0,70.0,60.0,60.0,55.0,...,75.0,25.0,6.80,6.21,0.00,0.00,0.00,0.00,NaN,NaN
2,3.0,AIRIN REGI,70.0,68.0,70.0,47.0,85.0,53.0,85.0,95.0,...,35.0,100.0,7.89,7.08,7.35,6.87,7.67,7.11,NaN,NaN
3,4.0,AJO ANTONY,55.0,73.0,75.0,50.0,70.0,57.0,30.0,65.0,...,20.0,45.0,6.67,0.00,0.00,0.00,0.00,0.00,NaN,NaN


## Datset Preprocessing

In [3]:
df.shape

(61, 109)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Columns: 109 entries, ROLLNO to Unnamed: 108
dtypes: float64(79), int64(28), object(2)
memory usage: 52.1+ KB


### Known things about Dataset
* From the above dataset we can conclude that dataset contains some NaN values, So that dataset cleaning is required


In [5]:
df.isnull().sum()

ROLLNO             2
STUDENT_NAME       2
BE103_First        2
BE110_First        2
BE101-05_First     2
                  ..
SGPA_S4            1
SGPA_S5            1
SGPA_S6            1
Unnamed: 107      61
Unnamed: 108      60
Length: 109, dtype: int64

* Last two columns are unnamed and contains NaN values. We can't do any statistical operations on those data. So just drop those columns.

In [6]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head(4)

,ROLLNO,STUDENT_NAME,BE103_First,BE110_First,BE101-05_First,CS110_First,EE100_First,EE110_First,MA101_First,PH100_First,...,CS405_Second,CS407_Second,CS409_Seccond,ELECTIVE3_Second,SGPA_S1,SGPA_S2,SGPA_S3,SGPA_S4,SGPA_S5,SGPA_S6
0,1.0,ABHIN JOY J A,55.0,45.0,80.0,47.0,70.0,57.0,65.0,45.0,...,45.0,65.0,25.0,45.0,7.00,0.00,0.00,0.00,0.00,6.22
1,2.0,AHKIN JACOB,60.0,93.0,60.0,57.0,70.0,60.0,60.0,55.0,...,60.0,20.0,75.0,25.0,6.80,6.21,0.00,0.00,0.00,0.00
2,3.0,AIRIN REGI,70.0,68.0,70.0,47.0,85.0,53.0,85.0,95.0,...,75.0,70.0,35.0,100.0,7.89,7.08,7.35,6.87,7.67,7.11
3,4.0,AJO ANTONY,55.0,73.0,75.0,50.0,70.0,57.0,30.0,65.0,...,55.0,50.0,20.0,45.0,6.67,0.00,0.00,0.00,0.00,0.00


* We successfully removed Unnamed colums successfully from our dataset which contains NaN values.
* Now take a look a tail end of our dataset. Since some students dropped out the course. There is chance for inadequate data and hence NaN value on the tail end.

In [7]:
df.tail(5)

,ROLLNO,STUDENT_NAME,BE103_First,BE110_First,BE101-05_First,CS110_First,EE100_First,EE110_First,MA101_First,PH100_First,...,CS405_Second,CS407_Second,CS409_Seccond,ELECTIVE3_Second,SGPA_S1,SGPA_S2,SGPA_S3,SGPA_S4,SGPA_S5,SGPA_S6
56,56.0,UDAY KUMAR NAIR,50.0,50.0,80.0,53.0,30.0,40.0,40.0,30.0,...,90.0,45.0,40.0,30.0,7.33,7.46,0.0,6.70,7.09,0.00
57,57.0,VAISHNAVI S NAIR,60.0,57.0,80.0,43.0,75.0,47.0,55.0,95.0,...,50.0,50.0,30.0,40.0,6.96,6.79,6.5,6.57,6.93,0.00
58,58.0,VISHWASKRISHNA A R,45.0,50.0,75.0,47.0,50.0,47.0,60.0,90.0,...,85.0,60.0,70.0,70.0,NaN,NaN,0.0,7.63,7.35,7.22
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.0,45.0,65.0,40.0,NaN,NaN,0.0,0.00,6.65,0.00
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* We also want to remove those rows contain NaN(Not a Number) values

In [8]:
df = df.dropna()

In [9]:
df.tail()

,ROLLNO,STUDENT_NAME,BE103_First,BE110_First,BE101-05_First,CS110_First,EE100_First,EE110_First,MA101_First,PH100_First,...,CS405_Second,CS407_Second,CS409_Seccond,ELECTIVE3_Second,SGPA_S1,SGPA_S2,SGPA_S3,SGPA_S4,SGPA_S5,SGPA_S6
53,53.0,SRIYA SURESH,65.0,95.0,85.0,60.0,85.0,63.0,70.0,95.0,...,80.0,45.0,50.0,35.0,8.02,8.10,0.00,6.96,7.17,6.65
54,54.0,SUDHEESH K S,65.0,100.0,80.0,73.0,60.0,63.0,75.0,65.0,...,80.0,80.0,95.0,30.0,7.89,8.19,7.88,8.09,8.43,7.22
55,55.0,TITTU PAULOSE,70.0,95.0,70.0,63.0,90.0,63.0,70.0,100.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
56,56.0,UDAY KUMAR NAIR,50.0,50.0,80.0,53.0,30.0,40.0,40.0,30.0,...,90.0,45.0,40.0,30.0,7.33,7.46,0.00,6.70,7.09,0.00
57,57.0,VAISHNAVI S NAIR,60.0,57.0,80.0,43.0,75.0,47.0,55.0,95.0,...,50.0,50.0,30.0,40.0,6.96,6.79,6.50,6.57,6.93,0.00


In [10]:
# Check what is the shape of our dataset now. Earlier it was (61, 109).
# Now we droped two columns (so we can expect number of colums 107) and also droped some columns containing NaN values
df.shape

(58, 107)

In [11]:
columns = list(df.columns.values)
print("Columns are: ", columns)

Columns are:  ['ROLLNO', 'STUDENT_NAME', 'BE103_First', 'BE110_First', 'BE101-05_First', 'CS110_First', 'EE100_First', 'EE110_First', 'MA101_First', 'PH100_First', 'PH110_First', 'BE103_Second', 'BE110_second', 'BE101-05_Second', 'EE100_Second', 'MA101_Second', 'PH100_Second', 'BE100_First', 'BE102_First', 'CS100_First', 'CS120_First', 'CY100_First', 'CY110_First', 'EC100_First', 'EC110_First', 'MA102_First', 'BE100_Second', 'BE102_Second', 'CS100_Second', 'CY100_Second', 'EC100_Second', 'MA102_second', 'CS201_First', 'CS203_First', 'CS205_First', 'CS207_First', 'CS231_First', 'CS233_First', 'HS210_First', 'MA201_First', 'CS201_Second', 'CS203_Second', 'CS205_Second', 'CS207_Second', 'HS210_Second', 'MA201_Second', 'CS202_First', 'CS204_First', 'CS206_First', 'CS208_First', 'CS232_First', 'CS234_First', 'HS200_First', 'MA202_First', 'CS202_Second', 'CS204_Second', 'CS206_Second', 'CS208_Second', 'HS200_Second', 'MA202_Second', 'CS301_First', 'CS303_First', 'CS305_First', 'CS307_First',

* We only want some meaningful numbers which have more correlation to our marks. So we have to drop Name and Roll Number of student which has no relationship with their performance. 

In [12]:
df.drop('ROLLNO', axis=1, inplace=True) # delete column without reassign dataframe
df.drop('STUDENT_NAME', axis=1, inplace=True)
df.head()

,BE103_First,BE110_First,BE101-05_First,CS110_First,EE100_First,EE110_First,MA101_First,PH100_First,PH110_First,BE103_Second,...,CS405_Second,CS407_Second,CS409_Seccond,ELECTIVE3_Second,SGPA_S1,SGPA_S2,SGPA_S3,SGPA_S4,SGPA_S5,SGPA_S6
0,55.0,45.0,80.0,47.0,70.0,57.0,65.0,45.0,73.0,60.0,...,45.0,65.0,25.0,45.0,7.00,0.00,0.00,0.00,0.00,6.22
1,60.0,93.0,60.0,57.0,70.0,60.0,60.0,55.0,60.0,60.0,...,60.0,20.0,75.0,25.0,6.80,6.21,0.00,0.00,0.00,0.00
2,70.0,68.0,70.0,47.0,85.0,53.0,85.0,95.0,93.0,65.0,...,75.0,70.0,35.0,100.0,7.89,7.08,7.35,6.87,7.67,7.11
3,55.0,73.0,75.0,50.0,70.0,57.0,30.0,65.0,47.0,40.0,...,55.0,50.0,20.0,45.0,6.67,0.00,0.00,0.00,0.00,0.00
4,65.0,65.0,60.0,63.0,55.0,57.0,45.0,70.0,87.0,45.0,...,60.0,30.0,60.0,40.0,7.07,7.10,0.00,0.00,6.83,0.00


* Since these marks contains result of two series tests, we are taking too much data in order to fed to our algorithm. So just take average marks of two tests

In [13]:
new_df = pd.DataFrame()

In [14]:
new_df['BE103_avg'] = df[['BE103_First', 'BE103_Second']].mean(axis=1)
new_df['BE101-05_avg'] = df[['BE101-05_First', 'BE101-05_Second']].mean(axis=1)
new_df['BE110_avg'] = df[['BE110_First', 'BE110_second']].mean(axis=1)
new_df['CS110_avg'] = df[['CS110_First']] # Since not found corresponding second series examination
new_df['EE100_avg'] = df[['EE100_First', 'EE100_Second']].mean(axis=1)
new_df['EE110_avg'] = df[['EE110_First']]
new_df['MA101_avg'] = df[['MA101_First', 'MA101_Second']].mean(axis=1)
new_df['PH100_avg'] = df[['PH100_First', 'PH100_Second']].mean(axis=1)
new_df['PH110_avg'] = df[['PH110_First']]
new_df['BE100_avg'] = df[['BE100_First', 'BE100_Second']].mean(axis=1)
new_df['BE102_avg'] = df[['BE102_First', 'BE102_Second']].mean(axis=1)
new_df['CS100_avg'] = df[['CS100_First', 'CS100_Second']].mean(axis=1)
new_df['CS120_avg'] = df[['CS120_First']]
new_df['CY100_avg'] = df[['CY100_First', 'CY100_Second']].mean(axis=1)
new_df['CY110_avg'] = df[['CY110_First']]
new_df['EC100_avg'] = df[['EC100_First', 'EC100_Second']].mean(axis=1)
new_df['EC110_avg'] = df[['EC110_First']]
new_df['MA102_avg'] = df[['MA102_First', 'MA102_second']].mean(axis=1)
new_df['CS201_avg'] = df[['CS201_First', 'CS201_Second']].mean(axis=1)
new_df['CS203_avg'] = df[['CS203_First', 'CS203_Second']].mean(axis=1)
new_df['CS205_avg'] = df[['CS205_First', 'CS205_Second']].mean(axis=1)
new_df['CS207_avg'] = df[['CS207_First', 'CS207_Second']].mean(axis=1)
new_df['CS231_avg'] = df[['CS231_First']]
new_df['CS233_avg'] = df[['CS233_First']]
new_df['CS233_avg'] = df[['CS233_First']]
new_df['HS210_avg'] = df[['HS210_First', 'HS210_Second']].mean(axis=1)
new_df['MA201_avg'] = df[['MA201_First', 'MA201_Second']].mean(axis=1)
new_df['CS202_avg'] = df[['CS202_First', 'CS202_Second']].mean(axis=1)
new_df['CS204_avg'] = df[['CS204_First', 'CS204_Second']].mean(axis=1)
new_df['CS206_avg'] = df[['CS206_First', 'CS206_Second']].mean(axis=1)
new_df['CS208_avg'] = df[['CS208_First', 'CS208_Second']].mean(axis=1)
new_df['CS232_avg'] = df[['CS232_First']]
new_df['CS234_avg'] = df[['CS234_First']]
new_df['HS200_avg'] = df[['HS200_First', 'HS200_Second']].mean(axis=1)
new_df['MA202_avg'] = df[['MA202_First', 'MA202_Second']].mean(axis=1)
new_df['CS301_avg'] = df[['CS301_First', 'CS301_Second']].mean(axis=1)
new_df['CS303_avg'] = df[['CS303_First', 'CS303_Second']].mean(axis=1)
new_df['CS305_avg'] = df[['CS305_First', 'CS305_Second']].mean(axis=1)
new_df['CS307_avg'] = df[['CS307_First', 'CS307_Second']].mean(axis=1)
new_df['CS309_avg'] = df[['CS309_First', 'CS309_Second']].mean(axis=1)
new_df['CS331_avg'] = df[['CS331_First']]
new_df['CS333_avg'] = df[['CS333_First']]
new_df['ELECTIVE1_avg'] = df[['ELECTIVE1_First', 'ELECTIVE1_Second']].mean(axis=1)
new_df['CS302_avg'] = df[['CS302_First', 'CS302_Second']].mean(axis=1)
new_df['CS304_avg'] = df[['CS304_First', 'CS304_Second']].mean(axis=1)
new_df['CS306_avg'] = df[['CS306_First', 'CS306_Second']].mean(axis=1)
new_df['CS308_avg'] = df[['CS308_First', 'CS308_Second']].mean(axis=1)
new_df['CS332_avg'] = df[['CS332_First']]
new_df['CS334_avg'] = df[['CS334_First']]
new_df['HS300_avg'] = df[['HS300_First', 'HS300_Second']].mean(axis=1)
new_df['ELECTIVE2_avg'] = df[['ELECTIVE2_First', 'ELECTIVE2_Second']].mean(axis=1)
new_df['CS401_avg'] = df[['CS401_First', 'CS401_Second']].mean(axis=1)
new_df['CS403_avg'] = df[['CS403_First', 'CS403_Second']].mean(axis=1)
new_df['CS405_avg'] = df[['CS405_First', 'CS405_Second']].mean(axis=1)
new_df['CS407_avg'] = df[['CS407_First', 'CS407_Second']].mean(axis=1)
new_df['CS409_avg'] = df[['CS409_First', 'CS409_Seccond']].mean(axis=1)
new_df['CS431_avg'] = df[['CS431_First']]
new_df['ELECTIVE3_avg'] = df[['ELECTIVE3_First', 'ELECTIVE3_Second']].mean(axis=1)
new_df['SGPA_S1'] = df[['SGPA_S1']]
new_df['SGPA_S2'] = df[['SGPA_S2']]
new_df['SGPA_S3'] = df[['SGPA_S3']]
new_df['SGPA_S4'] = df[['SGPA_S4']]
new_df['SGPA_S5'] = df[['SGPA_S5']]
new_df['SGPA_S6'] = df[['SGPA_S6']]
new_df.head(1)

,BE103_avg,BE101-05_avg,BE110_avg,CS110_avg,EE100_avg,EE110_avg,MA101_avg,PH100_avg,PH110_avg,BE100_avg,...,CS407_avg,CS409_avg,CS431_avg,ELECTIVE3_avg,SGPA_S1,SGPA_S2,SGPA_S3,SGPA_S4,SGPA_S5,SGPA_S6
0,57.5,75.0,52.5,47.0,72.5,57.0,50.0,42.5,73.0,45.0,...,70.0,42.5,60.0,32.5,7.0,0.0,0.0,0.0,0.0,6.22


In [16]:
new_df.to_csv('dataset/preprocessed_data.csv')